# [Weaviate 连接](https://weaviate.io/developers/weaviate/connections/connect-local)

## 安装 Weaviate 客户端
使用 pip 安装 Weaviate 客户端：

```shell
pip install -U weaviate-client   

```

## 编写测试脚本
创建一个test.py文件，内容如下

### 连接到本地部署的 Weaviate 

#### 没启用身份验证

In [ ]:
## V3

import weaviate

# Create the client
client = weaviate.Client(
    url="http://localhost:8080"
)

print(client.is_ready())

## V4

import weaviate

client = weaviate.connect_to_local()

print(client.is_ready())


In [ ]:
### 改变端口
### V3
import weaviate

# Create the client
client = weaviate.Client(
    url="http://127.0.0.1:8080"
    # The v3 client doesn't use the gRPC port
)

print(client.is_ready())

### V4
import weaviate

client = weaviate.connect_to_local(
    host="127.0.0.1",  # Use a string to specify the host
    port=8080,
    grpc_port=50051,
)

print(client.is_ready())

In [ ]:
import weaviate
from weaviate.auth import AuthApiKey

# 连接到本地部署的 Weaviate
client = weaviate.Client(
    url="http://127.0.0.1:8080",
    auth_client_secret=AuthApiKey("WVF5YThaHlkYwhGUSmCRgsX3tD5ngdN8pkih")
)

# 检查连接是否成功
print(client.is_ready())


### 已启用身份验证

In [ ]:
### V3 版本
# Set this environment variable
# WEAVIATE_API_KEY   your Weaviate instance API key

import os
import weaviate

# Create the client
client = weaviate.Client(
    url="http://localhost:8080",
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY")),
)

print(client.is_ready())

In [ ]:
#### V4 版本
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

client = weaviate.connect_to_local(
    auth_credentials=Auth.api_key(weaviate_api_key)
)

print(client.is_ready())

assert client.is_ready()

### OIDC身份验证
有关使用 OpenID Connect (OIDC) 进行身份验证的详细信息，[请参阅OIDC 配置。](https://weaviate.io/developers/weaviate/configuration/authentication#oidc)

有关更多客户端示例，[请参阅OIDC 身份验证](https://weaviate.io/developers/weaviate/connections/connect-custom#oidc-authentication)。

In [ ]:
### V3
# Set this environment variable
# COHERE_API_KEY     your Cohere API key

import os
import weaviate

client = weaviate.Client(
    url="http://localhost:8080",
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")
    }
)

print(client.is_ready())

In [ ]:
### V4
import os
import weaviate

# Best practice: store your credentials in environment variables
cohere_api_key = os.environ["COHERE_API_KEY"]

client = weaviate.connect_to_local(
    headers={
        "X-Cohere-Api-Key": cohere_api_key
    }
)

print(client.is_ready())

### gRPC 超时
Python 客户端 v4 和 TypeScript 客户端 v3 使用gRPC。 gRPC 协议对网络延迟敏感。 如果遇到连接超时，请调整初始化、查询和插入的超时值。

In [ ]:
### V4
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=AdditionalConfig(
        timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
    )
)

print(client.is_ready())

## 自定义连接
Python客户端 v4和TypeScript 客户端 v3为常见连接类型提供了辅助方法。它们还提供了自定义方法，以备您需要额外的连接配置时使用。


In [ ]:
### V4版本

# 或者自定义连接
import weaviate, os
from weaviate.classes.init import Auth
from weaviate.config import AdditionalConfig, Timeout

# Best practice: store your credentials in environment variables
http_host = os.environ["WCD_HTTP_HOST"]
grpc_host = os.environ["WCD_GRPC_HOST"]
weaviate_api_key = os.environ["WCD_DEMO_RO_KEY"]

client = weaviate.connect_to_custom(
    http_host=http_host,        # Hostname for the HTTP API connection
    http_port=443,              # Default is 80, WCD uses 443
    http_secure=True,           # Whether to use https (secure) for the HTTP API connection
    grpc_host=grpc_host,        # Hostname for the gRPC API connection
    grpc_port=443,              # Default is 50051, WCD uses 443
    grpc_secure=True,           # Whether to use a secure channel for the gRPC API connection
    auth_credentials=Auth.api_key(weaviate_api_key),    # API key for authentication
)


# 检查连接是否成功
print(client.is_ready())

# 关闭连接
print(client.close())


## 嵌入式 Weaviate
嵌入式 Weaviate 是一种部署模型，它从您的应用程序代码而不是从独立的 Weaviate 服务器安装运行 Weaviate 实例。

当 Embedded Weaviate 首次启动时，它会在您设置的位置创建一个永久数据存储persistence_data_path。当您的客户端退出时，Embedded Weaviate 实例也会退出，但数据会保留下来。

客户端下次运行时，会启动一个新的嵌入式 Weaviate 实例。新的嵌入式 Weaviate 实例会使用数据存储区中保存的数据。

有关使用嵌入式 Weaviate 的更多详细信息，请参阅嵌入式 Weaviate。

In [ ]:
#### V3
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options=EmbeddedOptions()
)


In [ ]:
### V4
import weaviate
import os

# Best practice: store your credentials in environment variables
openai_api_key = os.environ["OPENAI_API_KEY"]

client = weaviate.connect_to_embedded(
    version="1.26.1",
    headers={
        "X-OpenAI-Api-Key": openai_api_key
    },
)